In [2]:
import ROOT
import uproot
import numpy  as np
import pandas as pd

Welcome to JupyROOT 6.22/00


In [3]:
ROOT.gROOT.SetBatch(True)


# Declare a human-readable label for each variable
labels = {
        "pt_1": "Muon p_{T} / GeV",
        "pt_2": "Tau p_{T} / GeV",
        "eta_1": "Muon #eta",
        "eta_2": "Tau #eta",
        "phi_1": "Muon #phi",
        "phi_2": "Tau #phi",
        "pt_met": "Missing p_{T} / GeV",
        "phi_met": "Missing p_{T} (#phi)",
        "q_1": "Muon charge",
        "q_2": "Tau charge",
        "iso_1": "Muon isolation",
        "iso_2": "Tau isolation",
        "m_1": "Muon mass / GeV",
        "m_2": "Tau mass / GeV",
        "mt_1": "Muon transverse mass / GeV",
        "mt_2": "Tau transverse mass / GeV",
        "dm_2": "Tau decay mode",
        "m_vis": "Visible di-tau mass / GeV",
        "pt_vis": "Visible di-tau p_{T} / GeV",
        "mjj": "Di-jet mass / GeV",
        "ptjj": "Di-jet p_{T} / GeV",
        "jdeta": "Di-jet #Delta#eta",
        "jpt_1": "Leading jet p_{T} / GeV",
        "jpt_2": "Trailing jet p_{T} / GeV",
        "jeta_1": "Leading jet #eta",
        "jeta_2": "Trailing jet #eta",
        "jphi_1": "Leading jet #phi",
        "jphi_2": "Trailing jet #phi",
        "jm_1": "Leading jet mass / GeV",
        "jm_2": "Trailing jet mass / GeV",
        "jbtag_1": "Leading jet b-tag / GeV",
        "jbtag_2": "Trailing jet b-tag / GeV",
        "npv": "Number of primary vertices",
        "njets": "Number of jets",
        }


# Specify the color for each process
colors = {
        "ggH": ROOT.TColor.GetColor("#BF2229"),
        "qqH": ROOT.TColor.GetColor("#00A88F"),
        "TT": ROOT.TColor.GetColor(155, 152, 204),
        "W": ROOT.TColor.GetColor(222, 90, 106),
        "QCD":  ROOT.TColor.GetColor(250, 202, 255),
        "ZLL": ROOT.TColor.GetColor(100, 192, 232),
        "ZTT": ROOT.TColor.GetColor(248, 206, 104),
        }


# Retrieve a histogram from the input file based on the process and the variable
# name
def getHistogram(tfile, name, variable, tag=""):
    name = "{}_{}{}".format(name, variable, tag)
    h = tfile.Get(name)
    if not h:
        raise Exception("Failed to load histogram {}.".format(name))
    return h

In [4]:
# Main function of the plotting step
#
# The major part of the code below is dedicated to define a nice-looking layout.
# The interesting part is the combination of the histograms to the QCD estimation.
# There, we take the data histogram from the control region and subtract all known
# processes defined in simulation and define the remaining part as QCD. Then,
# this shape is extrapolated into the signal region with a scale factor.

# Styles
ROOT.gStyle.SetOptStat(0)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(ROOT.kWhite)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(600)
ROOT.gStyle.SetCanvasDefX(0)
ROOT.gStyle.SetCanvasDefY(0)

ROOT.gStyle.SetPadTopMargin(0.08)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.16)
ROOT.gStyle.SetPadRightMargin(0.05)

ROOT.gStyle.SetHistLineColor(1)
ROOT.gStyle.SetHistLineStyle(0)
ROOT.gStyle.SetHistLineWidth(1)
ROOT.gStyle.SetEndErrorSize(2)
ROOT.gStyle.SetMarkerStyle(20)

ROOT.gStyle.SetOptTitle(0)
ROOT.gStyle.SetTitleFont(42)
ROOT.gStyle.SetTitleColor(1)
ROOT.gStyle.SetTitleTextColor(1)
ROOT.gStyle.SetTitleFillColor(10)
ROOT.gStyle.SetTitleFontSize(0.05)

ROOT.gStyle.SetTitleColor(1, "XYZ")
ROOT.gStyle.SetTitleFont(42, "XYZ")
ROOT.gStyle.SetTitleSize(0.05, "XYZ")
ROOT.gStyle.SetTitleXOffset(1.00)
ROOT.gStyle.SetTitleYOffset(1.60)

ROOT.gStyle.SetLabelColor(1, "XYZ")
ROOT.gStyle.SetLabelFont(42, "XYZ")
ROOT.gStyle.SetLabelOffset(0.007, "XYZ")
ROOT.gStyle.SetLabelSize(0.04, "XYZ")

ROOT.gStyle.SetAxisColor(1, "XYZ")
ROOT.gStyle.SetStripDecimals(True)
ROOT.gStyle.SetTickLength(0.03, "XYZ")
ROOT.gStyle.SetNdivisions(510, "XYZ")
ROOT.gStyle.SetPadTickX(1)
ROOT.gStyle.SetPadTickY(1)

ROOT.gStyle.SetPaperSize(20., 20.)
ROOT.gStyle.SetHatchesLineWidth(5)
ROOT.gStyle.SetHatchesSpacing(0.05)

ROOT.TGaxis.SetExponentOffset(-0.08, 0.01, "Y")

In [6]:
#Loading in the required variables
sig_df = pd.read_pickle('../DNN_ips/SUSY_Mstop_175_mchi_7_5.pkl')
tt_df  = pd.read_pickle('../DNN_ips/ttbar.pkl')
vv_df  = pd.read_pickle('../DNN_ips/VV.pkl')
st_df  = pd.read_pickle('../DNN_ips/Single_top.pkl')
sat_df = pd.read_pickle('../DNN_ips/Single_antitop.pkl')

In [8]:
vv_lep_pt    = np.array(vv_df['sublepton_pt'])
tt_lep_pt    = np.array(tt_df['sublepton_pt'])
st_lep_pt    = np.array(st_df['sublepton_pt'])
sat_lep_pt   = np.array(sat_df['sublepton_pt'])
sig_lep_pt   = np.array(sig_df['sublepton_pt'])

In [17]:
xlim  = 300
nbins = 25

h_vv_l_pt   = ROOT.TH1F('vv_l_pt'  , 'vv_l_pt'  , nbins, 0, xlim)
h_st_l_pt   = ROOT.TH1F('st_l_pt'  , 'st_l_pt'  , nbins, 0, xlim)
h_sat_l_pt  = ROOT.TH1F('sat_l_pt' , 'sat_l_pt' , nbins, 0, xlim)
h_tt_l_pt   = ROOT.TH1F('tt_l_pt'  , 'tt_l_pt'  , nbins, 0, xlim)
h_SUSY_l_pt = ROOT.TH1F('SUSY_l_pt', 'SUSY_l_pt', nbins, 0, xlim)

Warning in <TROOT::Append>: Replacing existing TH1: vv_l_pt (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: st_l_pt (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sat_l_pt (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: tt_l_pt (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: SUSY_l_pt (Potential memory leak).


In [18]:
for pt in vv_lep_pt :
    if(pt > xlim) :
        h_vv_l_pt.Fill(xlim-1)
    else :
        h_vv_l_pt.Fill(pt)

In [19]:
for pt in tt_lep_pt :
    if(pt > xlim) :
        h_tt_l_pt.Fill(xlim -1)
    else :
        h_tt_l_pt.Fill(pt)

In [20]:
for pt in st_lep_pt :
    if(pt > xlim) :
        h_st_l_pt.Fill(xlim -1)
    else :
        h_st_l_pt.Fill(pt)

In [21]:
for pt in sat_lep_pt :
    if(pt > xlim) :
        h_sat_l_pt.Fill(xlim -1)
    else :
        h_sat_l_pt.Fill(pt)

In [22]:
for pt in sig_lep_pt :
    if(pt > xlim) :
        h_SUSY_l_pt.Fill(xlim -1)
    else :
        h_SUSY_l_pt.Fill(pt)

In [23]:
# Draw histograms
lumi      = 3000
scale_tt  = 17.994815
h_tt_l_pt.Scale(scale_tt)

scale_l_pt  = 8.706901
h_st_l_pt.Scale(scale_l_pt)

scale_sat = 8.706901
h_sat_l_pt.Scale(scale_sat)

scale_vv  = 2.580743
h_vv_l_pt.Scale(scale_vv)

scale_SUSY= 14.389516
h_SUSY_l_pt.Scale(scale_SUSY)

In [25]:
h_SUSY_l_pt.SetLineColor(colors["qqH"])
h_SUSY_l_pt.SetLineWidth(3)

for x, l in [(h_tt_l_pt, "W"), (h_st_l_pt, "ZTT"), (h_sat_l_pt, "ZTT"), (h_vv_l_pt, "ZLL")]:
    x.SetLineWidth(0)
    x.SetFillColor(colors[l])

stack = ROOT.THStack("", "")
for x in [h_vv_l_pt, h_sat_l_pt, h_st_l_pt, h_tt_l_pt, h_SUSY_l_pt]:
    stack.Add(x)

c = ROOT.TCanvas("", "", 800, 800)
stack.Draw("hist")

stack.GetXaxis().SetTitle('Sub-leading lepton p_{T}[GeV]')
stack.GetYaxis().SetTitle("N_{Events} / bin")
stack.SetMaximum(stack.GetMaximum() * 1.1)
stack.SetMinimum(100.0)
#ROOT.gPad.SetLogy()

# Add legend
legend = ROOT.TLegend(0.4, 0.73, 0.80, 0.88)
legend.SetNColumns(1)
legend.AddEntry(h_tt_l_pt , "t#bar{t}", "f")
legend.AddEntry(h_st_l_pt , "Single Top", "f")
legend.AddEntry(h_vv_l_pt , "Diboson", "f")
legend.AddEntry(h_SUSY_l_pt,"SUSY m_{ #tilde{t}} = 175 GeV m_{#chi0} = 7.5 GeV", "l")
legend.SetBorderSize(0)
legend.Draw()

# Add title
latex = ROOT.TLatex()
latex.SetNDC()
latex.SetTextSize(0.04)
latex.SetTextFont(42)
latex.DrawLatex(0.67, 0.935, "3000 fb^{{-1}} (14 TeV)".format(lumi))
latex.DrawLatex(0.16, 0.935, "#bf{ CMS Projection}")
ROOT.gStyle.SetLegendTextSize(0.03)

# Save
c.SaveAs("slpt_w_of_bin.pdf")

Info in <TCanvas::Print>: pdf file slpt_w_of_bin.pdf has been created


In [246]:
#Loading in the required variables
vv_llbar_dphi    = np.array(vv_df['bbbar_dphi'])
tt_llbar_dphi    = np.array(tt_df['bbbar_dphi'])
st_llbar_dphi    = np.array(st_df['bbbar_dphi'])
sat_llbar_dphi   = np.array(sat_df['bbbar_dphi'])
sig_llbar_dphi   = np.array(sig_df['bbbar_dphi'])

In [247]:
x_low       = 0
x_high      = np.pi
n_bins      = 10

h_vv_llbar_dphi   = ROOT.TH1F('vv_llbar_dphi'  , 'vv_llbar_dphi'  , n_bins, x_low, x_high)
h_st_llbar_dphi   = ROOT.TH1F('st_llbar_dphi'  , 'st_llbar_dphi'  , n_bins, x_low, x_high)
h_sat_llbar_dphi  = ROOT.TH1F('sat_llbar_dphi' , 'sat_llbar_dphi' , n_bins, x_low, x_high)
h_tt_llbar_dphi   = ROOT.TH1F('tt_llbar_dphi'  , 'tt_llbar_dphi'  , n_bins, x_low, x_high)
h_SUSY_llbar_dphi = ROOT.TH1F('SUSY_llbar_dphi', 'SUSY_llbar_dphi', n_bins, x_low, x_high)

Warning in <TROOT::Append>: Replacing existing TH1: vv_llbar_dphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: st_llbar_dphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sat_llbar_dphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: tt_llbar_dphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: SUSY_llbar_dphi (Potential memory leak).


In [248]:
for dphi in vv_llbar_dphi :
    h_vv_llbar_dphi.Fill(dphi)
    
for dphi in tt_llbar_dphi :
    h_tt_llbar_dphi.Fill(dphi)
    
for dphi in st_llbar_dphi :
    h_st_llbar_dphi.Fill(dphi)

for dphi in sat_llbar_dphi :
    h_sat_llbar_dphi.Fill(dphi)
    
for dphi in sig_llbar_dphi :
    h_SUSY_llbar_dphi.Fill(dphi)

In [249]:
# Draw histograms
lumi  = 3000
scale_tt  = 17.994815
h_tt_llbar_dphi.Scale(scale_tt)

scale_st  = 8.706901
h_st_llbar_dphi.Scale(scale_st)

scale_sat = 8.706901
h_sat_llbar_dphi.Scale(scale_sat)

scale_vv  = 2.580743
h_vv_llbar_dphi.Scale(scale_vv)

scale_SUSY= 14.389516
h_SUSY_llbar_dphi.Scale(scale_SUSY)

In [250]:
h_SUSY_llbar_dphi.SetLineColor(colors["qqH"])
h_SUSY_llbar_dphi.SetLineWidth(3)

for x, l in [(h_tt_llbar_dphi, "W"), (h_st_llbar_dphi, "ZTT"), (h_sat_llbar_dphi, "ZTT"), (h_vv_llbar_dphi, "ZLL")]:
    x.SetLineWidth(0)
    x.SetFillColor(colors[l])

stack = ROOT.THStack("", "")
for x in [h_sat_llbar_dphi, h_st_llbar_dphi, h_vv_llbar_dphi, h_tt_llbar_dphi, h_SUSY_llbar_dphi]:
    stack.Add(x)

c = ROOT.TCanvas("", "", 800, 800)
stack.Draw("hist")

stack.GetXaxis().SetTitle('\Delta \phi_{jj}')
stack.GetYaxis().SetTitle("N_{Events} /  bin")
stack.SetMaximum(stack.GetMaximum()*1.3 )
#stack.SetMinimum(100.0)
#ROOT.gStyle.SetTitleFontSize(0.05)


# Add legend
legend = ROOT.TLegend(0.27, 0.73, 0.67, 0.88)
legend.SetNColumns(1)
legend.AddEntry(h_tt_llbar_dphi , "t#bar{t}", "f")
legend.AddEntry(h_vv_llbar_dphi , "Diboson", "f")
legend.AddEntry(h_st_llbar_dphi , "Single Top", "f")
legend.AddEntry(h_SUSY_llbar_dphi, "SUSY m_{ #tilde{t}} = 175 GeV m_{#chi0} = 7.5 GeV", "l")
legend.SetBorderSize(0)
legend.Draw()

# Add title
latex = ROOT.TLatex()
latex.SetNDC()
latex.SetTextSize(0.04)
latex.SetTextFont(42)
latex.DrawLatex(0.67, 0.935, "3000 fb^{{-1}} (14 TeV)".format(lumi))
latex.DrawLatex(0.16, 0.935, "#bf{ CMS Projection}")
ROOT.gStyle.SetLegendTextSize(0.03)

# Save
c.SaveAs("bbbar_dphi.pdf")

Info in <TCanvas::Print>: pdf file bbbar_dphi.pdf has been created


In [7]:
sig_df.keys

<bound method NDFrame.keys of          lepton_pt  lepton_eta  lepton_phi  sublepton_pt  sublepton_eta  \
0        54.800117    1.631996    0.838088     30.457233      -0.072351   
1        62.871208   -0.502981    1.041450     53.695518      -0.962942   
2        77.377106    0.397918   -0.471417     44.754673       0.134825   
3        33.959000    1.905253    1.452651     26.326990       0.545058   
4        58.220764    0.724194    1.281943     35.121616      -0.592697   
...            ...         ...         ...           ...            ...   
125934   57.198299    0.769189   -0.495362     51.997704       0.167035   
125935   40.453503    1.226450    2.385078     26.996359       2.329891   
125936   56.961548    1.253774    2.691810     24.658413       0.200721   
125937  249.689850    0.172857   -0.140644     61.737934       0.313576   
125938  102.404465    1.678013    2.461559     28.256666      -0.343879   

        sublepton_phi     ljet_pt  ljet_eta  ljet_phi    sljet_pt  sl